In [1]:
import numpy as np
import theano
import theano.tensor as T

In [2]:
rng = np.random

N = 400                                   # training sample size
feats = 784                               # number of input variables

In [3]:
# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

In [4]:
# Declare Theano symbolic variables
x = T.dmatrix("x")
y = T.dvector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
print(w.get_value())
print(b.get_value())

Initial model:
[  4.55410451e-01  -2.70005391e-01   1.87727011e+00  -3.63827155e-01
  -3.22880207e-01   1.34204286e+00  -1.59536746e-01  -1.11299311e-01
   7.87786514e-01  -1.13504167e-01  -5.28206269e-01   5.23545768e-01
   3.17327639e-01  -8.87557727e-02  -2.90002017e-01  -1.41387976e+00
   4.18771576e-01   9.73481265e-01  -8.77196515e-01   5.61129875e-01
  -7.41142863e-01  -1.33148704e+00   7.98021849e-01  -3.90520359e-01
  -7.98975064e-03   6.20739995e-01  -8.36332948e-03  -3.34166509e-01
  -7.77777085e-01  -3.18903566e-01   5.70236176e-01   2.05816332e+00
  -3.31708805e-01   1.05324494e+00   5.68344606e-01  -5.03081154e-01
   6.65837335e-01   1.47149902e+00  -8.28911830e-01  -1.38517262e+00
  -4.02989052e-01   3.11536142e-01  -1.65980366e+00  -1.36908273e+00
   1.95761324e-01   4.73598592e-01   1.69075566e+00   3.67974326e-01
  -6.97356462e-01  -1.98522990e+00  -5.15535413e-01   4.45619056e-01
  -7.12212178e-01  -1.83455898e+00  -7.98862258e-01   3.30009017e-01
  -1.29656785e+00  

In [5]:
# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

In [6]:
# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent, cost],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

In [7]:
# Train
prev_cost = np.inf
print(prev_cost - cost)
for i in range(training_steps):
    pred, err, cost = train(D[0], D[1])
    acc = np.sum(pred == D[1]) / N
    if prev_cost - cost < 0.001:
        break
    if i % 50 == 0:
        print('%d\tcost:%f\tacc:%f' % (i,cost,acc))
    prev_cost = cost    

Elemwise{sub,no_inplace}.0
0	cost:18.561109	acc:0.530000
50	cost:11.573083	acc:0.637500
100	cost:7.375502	acc:0.762500
150	cost:4.894575	acc:0.865000
200	cost:3.460422	acc:0.912500
250	cost:2.571702	acc:0.970000
300	cost:2.019750	acc:0.992500
350	cost:1.642096	acc:1.000000
400	cost:1.353900	acc:1.000000
450	cost:1.121167	acc:1.000000
500	cost:0.932005	acc:1.000000
550	cost:0.778073	acc:1.000000
600	cost:0.652843	acc:1.000000
650	cost:0.551051	acc:1.000000
700	cost:0.468410	acc:1.000000
750	cost:0.401413	acc:1.000000


In [8]:
print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
[ -5.98018414e-02   2.72092485e-02   2.24057298e-01  -1.16590068e-01
   1.26554913e-01   3.18684638e-01   3.00449038e-01  -2.45988380e-01
   5.06528465e-03  -2.32453733e-01  -8.60259833e-03   2.26788724e-01
   1.68481470e-01  -1.10951331e-01   1.00445283e-02  -1.84218157e-01
   5.28955670e-02   4.61989288e-02  -1.40731538e-01  -7.04631919e-02
  -7.36160617e-02  -1.38958150e-01   1.43345905e-02  -3.90259487e-02
   2.36084943e-01   1.20757148e-01   1.73300256e-01  -1.45053082e-01
  -2.27346936e-01   2.08669634e-01  -1.39835035e-01   2.76152086e-01
  -1.89000175e-02  -9.48081327e-02   2.59307498e-01  -2.16266611e-01
   4.34414570e-02   2.90296728e-01  -1.34464557e-01  -1.40343082e-01
  -5.64575666e-02   1.39197779e-01  -4.05703008e-01  -2.05482895e-01
  -1.25051031e-01   5.54052992e-02   2.67911950e-01  -1.22655016e-01
  -3.90244782e-02  -3.60036000e-02  -2.19156599e-01   1.55392091e-01
  -1.51971090e-02  -2.59565792e-01  -1.67040368e-02  -1.04732874e-01
  -2.28645497e-01   5